In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
import seaborn as sns
import os
import warnings
from google.colab import files

uploaded = files.upload()
# only need to run this cell once to install packages and install CSV files
# check files to see if BofA.csv, Goldman.csv, JPM.csv, and Rates.csv are present




Saving JPM.csv to JPM.csv
Saving Goldman.csv to Goldman.csv
Saving BofA.csv to BofA.csv
Saving Rates.csv to Rates.csv


In [ ]:
# this cell will run all analysis print out accuracy metrics
# graphs will be saved as pngs in files on the left with CSV files
warnings.filterwarnings("ignore")

def parse_number(x):
    if isinstance(x, str):
        x = x.replace(',', '').replace('$', '').replace('%', '').strip()
        if x.startswith('(') and x.endswith(')'):
            x = '-' + x[1:-1]
        try:
            return float(x)
        except ValueError:
            return np.nan
    elif isinstance(x, (int, float)):
        return x
    else:
        return np.nan

# Load and prepare bank data
def load_bank_data(filename):
    try:
        df = pd.read_csv(filename, thousands=',')
        print(f"Successfully read {filename}")
    except FileNotFoundError:
        print(f"File not found: {filename}")
        return None
    except pd.errors.ParserError as e:
        print(f"Parser error reading {filename}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error reading {filename}: {e}")
        return None

    df.columns = df.columns.str.strip()
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    if 'Pricing Date' not in df.columns:
        print("Column 'Pricing Date' not found in the DataFrame.")
        print("Available columns:", df.columns.tolist())
        return None

    try:
        df['Pricing Date'] = pd.to_datetime(df['Pricing Date'], format='%m-%d-%Y', errors='coerce')
        if df['Pricing Date'].isnull().any():
            print("Some 'Pricing Date' values could not be converted to datetime.")
            df = df.dropna(subset=['Pricing Date'])
    except Exception as e:
        print(f"Error converting 'Pricing Date' to datetime: {e}")
        return None

    if df.empty:
        print(f"All 'Pricing Date' values in {filename} could not be converted. Skipping this file.")
        return None

    df = df.set_index('Pricing Date')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    # Handle duplicate dates
    if df.index.duplicated().any():
        print("Duplicate dates found in index. Handling duplicates...")
        df = df[~df.index.duplicated(keep='first')]  # Remove duplicates

    df = df.asfreq('B')
    df = df.ffill()
    return df

def load_rates_data(filename):
    try:
        df = pd.read_csv(filename, thousands=',')
        print(f"Successfully read {filename}")
    except FileNotFoundError:
        print(f"File not found: {filename}")
        return None
    except pd.errors.ParserError as e:
        print(f"Parser error reading {filename}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error reading {filename}: {e}")
        return None

    df.columns = df.columns.str.strip()

    if 'DATE' not in df.columns:
        print("Column 'DATE' not found in the DataFrame.")
        print("Available columns:", df.columns.tolist())
        return None

    try:
        df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
        if df['DATE'].isnull().any():
            print("Some 'DATE' values could not be converted to datetime.")
            df = df.dropna(subset=['DATE'])
    except Exception as e:
        print(f"Error converting 'DATE' to datetime: {e}")
        return None

    if df.empty:
        print(f"All 'DATE' values in {filename} could not be converted. Skipping this file.")
        return None

    df = df.set_index('DATE')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df.asfreq('B')
    df = df.ffill()
    return df

# Perform and print ADF test results
def perform_stationarity_test(series, name):
    print(f"\n=== Stationarity Test for {name} ===")
    result = adfuller(series.dropna())
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')
    is_stationary = result[1] < 0.05
    if is_stationary:
        print(f"{name} is stationary.")
    else:
        print(f"{name} is non-stationary. Differencing is recommended.")
    return is_stationary


def extract_granger_pvalues(granger_result):
    p_values = {}
    for lag in granger_result.keys():
        test_result = granger_result[lag][0]['ssr_chi2test']
        p_values[lag] = test_result[1]
    return p_values
def analyze_bank_rates_relationship(bank_df, rates_df, bank_name):
    if bank_df is None or rates_df is None:
        print(f"Data for {bank_name} or interest rates is missing.")
        return None

    bank_df = bank_df.loc[:, ~bank_df.columns.str.contains('^Unnamed')]

    # Merge data
    merged_df = pd.merge(bank_df, rates_df, left_index=True, right_index=True, how='inner')
    merged_df = merged_df.dropna()

    if merged_df.empty:
        print(f"No overlapping dates between {bank_name} data and interest rates.")
        return None

    # Get price column name
    price_col_candidates = [col for col in merged_df.columns if 'Share Price' in col]
    if not price_col_candidates:
        print(f"No 'Share Price' column found for {bank_name}.")
        print("Available columns:", merged_df.columns.tolist())
        return None
    price_col = price_col_candidates[0]

    merged_df[price_col] = merged_df[price_col].apply(parse_number).astype(float)

    if 'MORTGAGE30US' not in merged_df.columns:
        print("'MORTGAGE30US' column not found in interest rate data.")
        print("Available columns in merged_df:", merged_df.columns.tolist())
        return None

    merged_df['MORTGAGE30US'] = merged_df['MORTGAGE30US'].apply(parse_number).astype(float)

    # Drop any rows with NaNs resulted from conversion
    merged_df = merged_df.dropna(subset=[price_col, 'MORTGAGE30US'])
    merged_df = merged_df.replace([np.inf, -np.inf], np.nan)
    merged_df = merged_df.dropna(subset=[price_col, 'MORTGAGE30US'])

    if merged_df.shape[0] < 10:
        print(f"Not enough data points after cleaning for {bank_name}.")
        return None

    print(f"Date range of merged_df: {merged_df.index.min()} to {merged_df.index.max()}")

    # Calculate correlation
    correlation = merged_df[price_col].corr(merged_df['MORTGAGE30US'])
    print(f"Correlation between {price_col} and MORTGAGE30US: {correlation}")

    # Perform Granger causality test
    num_obs = merged_df.shape[0]
    max_lag = min(5, num_obs - 2)
    if max_lag < 1:
        print(f"Insufficient observations for Granger causality test for {bank_name}.")
        granger_pvalues = None
    else:
        try:
            granger_result = grangercausalitytests(
                merged_df[[price_col, 'MORTGAGE30US']], maxlag=max_lag, verbose=False
            )
            granger_pvalues = extract_granger_pvalues(granger_result)
        except Exception as e:
            print(f"Error performing Granger causality test for {bank_name}: {e}")
            granger_pvalues = None

    # Store original series before differencing
    original_price_series = merged_df[price_col].copy()
    original_rate_series = merged_df['MORTGAGE30US'].copy()

    # Check for stationarity
    is_stationary_price = perform_stationarity_test(merged_df[price_col], f"{bank_name} Price")
    is_stationary_rate = perform_stationarity_test(merged_df['MORTGAGE30US'], "Interest Rate")

    # Initialize differencing order
    d_price = 0
    d_rate = 0

    # Apply differencing if necessary
    if not is_stationary_price:
        merged_df[price_col] = merged_df[price_col].diff()
        d_price += 1

    if not is_stationary_rate:
        merged_df['MORTGAGE30US'] = merged_df['MORTGAGE30US'].diff()
        d_rate += 1

    # Re-drop NA after differencing
    merged_df = merged_df.dropna()

    # Re-test for stationarity after differencing
    if not is_stationary_price:
        is_stationary_price = perform_stationarity_test(merged_df[price_col], f"{bank_name} Price After Differencing")
    if not is_stationary_rate:
        is_stationary_rate = perform_stationarity_test(merged_df['MORTGAGE30US'], "Interest Rate After Differencing")

    if not is_stationary_price or not is_stationary_rate:
        print(f"Series are still non-stationary after differencing for {bank_name}.")
        return None

    # Split data into training and testing sets
    train_size = int(len(merged_df) * 0.8)
    train_data = merged_df.iloc[:train_size]
    test_data = merged_df.iloc[train_size:]

    forecast_steps = len(test_data)
    actual = original_price_series.iloc[train_size:train_size + forecast_steps]

    # ARIMA Model with Exogenous Variables
    arima_order = (2, d_price, 2)
    arima_model = ARIMA(
        train_data[price_col],
        exog=train_data[['MORTGAGE30US']],
        order=arima_order
    )
    try:
        arima_results = arima_model.fit()
    except Exception as e:
        print(f"Error fitting ARIMA model for {bank_name}: {e}")
        return None

    # Forecasting with ARIMA (with exogenous variables)
    try:
        arima_forecast = arima_results.get_forecast(steps=forecast_steps, exog=test_data[['MORTGAGE30US']])
        arima_forecast_mean = arima_forecast.predicted_mean
        arima_conf_int = arima_forecast.conf_int()
        arima_conf_int.columns = ['lower', 'upper']
        arima_forecast_mean.index = test_data.index
        arima_conf_int.index = test_data.index

        # Reconstruct forecasted levels if differenced
        if d_price > 0:
            last_value = original_price_series.iloc[train_size - 1]
            arima_forecast_values = arima_forecast_mean.cumsum() + last_value
            arima_conf_int['lower'] = arima_conf_int['lower'].cumsum() + last_value
            arima_conf_int['upper'] = arima_conf_int['upper'].cumsum() + last_value
            arima_forecast_mean = arima_forecast_values

    except Exception as e:
        print(f"Error forecasting with ARIMA for {bank_name}: {e}")
        return None

    # Calculate ARIMA accuracy metrics (with exogenous variables)
    arima_mae = np.mean(np.abs(actual - arima_forecast_mean))
    arima_mse = np.mean((actual - arima_forecast_mean) ** 2)
    arima_rmse = np.sqrt(arima_mse)
    arima_mape = np.mean(np.abs((actual - arima_forecast_mean) / actual)) * 100

    arima_accuracy_metrics = {
        'ARIMA_MAE_with_exog': arima_mae,
        'ARIMA_MSE_with_exog': arima_mse,
        'ARIMA_RMSE_with_exog': arima_rmse,
        'ARIMA_MAPE_with_exog': arima_mape
    }

    # ARIMA Model without Exogenous Variables
    arima_model_no_exog = ARIMA(
        train_data[price_col],
        order=arima_order
    )
    try:
        arima_results_no_exog = arima_model_no_exog.fit()
    except Exception as e:
        print(f"Error fitting ARIMA model without exogenous variables for {bank_name}: {e}")
        return None

    # Forecasting with ARIMA (without exogenous variables)
    try:
        arima_forecast_no_exog = arima_results_no_exog.get_forecast(steps=forecast_steps)
        arima_forecast_mean_no_exog = arima_forecast_no_exog.predicted_mean
        arima_conf_int_no_exog = arima_forecast_no_exog.conf_int()
        arima_conf_int_no_exog.columns = ['lower', 'upper']
        arima_forecast_mean_no_exog.index = test_data.index
        arima_conf_int_no_exog.index = test_data.index

        # Reconstruct forecasted levels if differenced
        if d_price > 0:
            last_value = original_price_series.iloc[train_size - 1]
            arima_forecast_values_no_exog = arima_forecast_mean_no_exog.cumsum() + last_value
            arima_conf_int_no_exog['lower'] = arima_conf_int_no_exog['lower'].cumsum() + last_value
            arima_conf_int_no_exog['upper'] = arima_conf_int_no_exog['upper'].cumsum() + last_value
            arima_forecast_mean_no_exog = arima_forecast_values_no_exog

    except Exception as e:
        print(f"Error forecasting with ARIMA without exogenous variables for {bank_name}: {e}")
        return None

    # Calculate ARIMA accuracy metrics (without exogenous variables)
    arima_mae_no_exog = np.mean(np.abs(actual - arima_forecast_mean_no_exog))
    arima_mse_no_exog = np.mean((actual - arima_forecast_mean_no_exog) ** 2)
    arima_rmse_no_exog = np.sqrt(arima_mse_no_exog)
    arima_mape_no_exog = np.mean(np.abs((actual - arima_forecast_mean_no_exog) / actual)) * 100

    arima_accuracy_metrics_no_exog = {
        'ARIMA_MAE_no_exog': arima_mae_no_exog,
        'ARIMA_MSE_no_exog': arima_mse_no_exog,
        'ARIMA_RMSE_no_exog': arima_rmse_no_exog,
        'ARIMA_MAPE_no_exog': arima_mape_no_exog
    }

    # SARIMA Model with Exogenous Variables
    seasonal_order = (1, 0, 1, 5)
    sarima_model = SARIMAX(
        train_data[price_col],
        exog=train_data[['MORTGAGE30US']],
        order=(2, d_price, 2),
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    try:
        sarima_results = sarima_model.fit(disp=False)
    except Exception as e:
        print(f"Error fitting SARIMA model for {bank_name}: {e}")
        return None

    # Forecasting with SARIMA (with exogenous variables)
    try:
        sarima_forecast = sarima_results.get_forecast(steps=forecast_steps, exog=test_data[['MORTGAGE30US']])
        sarima_forecast_mean = sarima_forecast.predicted_mean
        sarima_conf_int = sarima_forecast.conf_int()
        sarima_conf_int.columns = ['lower', 'upper']
        sarima_forecast_mean.index = test_data.index
        sarima_conf_int.index = test_data.index

        # Reconstruct forecasted levels if differenced
        if d_price > 0:
            last_value = original_price_series.iloc[train_size - 1]
            sarima_forecast_values = sarima_forecast_mean.cumsum() + last_value
            sarima_conf_int['lower'] = sarima_conf_int['lower'].cumsum() + last_value
            sarima_conf_int['upper'] = sarima_conf_int['upper'].cumsum() + last_value
            sarima_forecast_mean = sarima_forecast_values

    except Exception as e:
        print(f"Error forecasting with SARIMA for {bank_name}: {e}")
        return None

    # Calculate SARIMA accuracy metrics (with exogenous variables)
    sarima_mae = np.mean(np.abs(actual - sarima_forecast_mean))
    sarima_mse = np.mean((actual - sarima_forecast_mean) ** 2)
    sarima_rmse = np.sqrt(sarima_mse)
    sarima_mape = np.mean(np.abs((actual - sarima_forecast_mean) / actual)) * 100

    sarima_accuracy_metrics = {
        'SARIMA_MAE_with_exog': sarima_mae,
        'SARIMA_MSE_with_exog': sarima_mse,
        'SARIMA_RMSE_with_exog': sarima_rmse,
        'SARIMA_MAPE_with_exog': sarima_mape
    }

    # SARIMA Model without Exogenous Variables
    sarima_model_no_exog = SARIMAX(
        train_data[price_col],
        order=(1, d_price, 1),
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    try:
        sarima_results_no_exog = sarima_model_no_exog.fit(disp=False)
    except Exception as e:
        print(f"Error fitting SARIMA model without exogenous variables for {bank_name}: {e}")
        return None

    # Forecasting with SARIMA (without exogenous variables)
    try:
        sarima_forecast_no_exog = sarima_results_no_exog.get_forecast(steps=forecast_steps)
        sarima_forecast_mean_no_exog = sarima_forecast_no_exog.predicted_mean
        sarima_conf_int_no_exog = sarima_forecast_no_exog.conf_int()
        sarima_conf_int_no_exog.columns = ['lower', 'upper']
        sarima_forecast_mean_no_exog.index = test_data.index
        sarima_conf_int_no_exog.index = test_data.index

        # Reconstruct forecasted levels if differenced
        if d_price > 0:
            last_value = original_price_series.iloc[train_size - 1]
            sarima_forecast_values_no_exog = sarima_forecast_mean_no_exog.cumsum() + last_value
            sarima_conf_int_no_exog['lower'] = sarima_conf_int_no_exog['lower'].cumsum() + last_value
            sarima_conf_int_no_exog['upper'] = sarima_conf_int_no_exog['upper'].cumsum() + last_value
            sarima_forecast_mean_no_exog = sarima_forecast_values_no_exog

    except Exception as e:
        print(f"Error forecasting with SARIMA without exogenous variables for {bank_name}: {e}")
        return None

    # Calculate SARIMA accuracy metrics (without exogenous variables)
    sarima_mae_no_exog = np.mean(np.abs(actual - sarima_forecast_mean_no_exog))
    sarima_mse_no_exog = np.mean((actual - sarima_forecast_mean_no_exog) ** 2)
    sarima_rmse_no_exog = np.sqrt(sarima_mse_no_exog)
    sarima_mape_no_exog = np.mean(np.abs((actual - sarima_forecast_mean_no_exog) / actual)) * 100

    sarima_accuracy_metrics_no_exog = {
        'SARIMA_MAE_no_exog': sarima_mae_no_exog,
        'SARIMA_MSE_no_exog': sarima_mse_no_exog,
        'SARIMA_RMSE_no_exog': sarima_rmse_no_exog,
        'SARIMA_MAPE_no_exog': sarima_mape_no_exog
    }

    # Holt-Winters Exponential Smoothing Model
    try:
        hw_model = ExponentialSmoothing(
            train_data[price_col],
            trend='add',
            seasonal='add',
            seasonal_periods=5  # Adjust seasonal_periods as needed
        ).fit()

        hw_forecast = hw_model.forecast(steps=forecast_steps)
        hw_forecast.index = test_data.index

        # Reconstruct forecasted levels if differenced
        if d_price > 0:
            last_value = original_price_series.iloc[train_size - 1]
            hw_forecast = hw_forecast.cumsum() + last_value

    except Exception as e:
        print(f"Error fitting Holt-Winters model for {bank_name}: {e}")
        return None

    # Calculate Holt-Winters accuracy metrics
    hw_mae = np.mean(np.abs(actual - hw_forecast))
    hw_mse = np.mean((actual - hw_forecast) ** 2)
    hw_rmse = np.sqrt(hw_mse)
    hw_mape = np.mean(np.abs((actual - hw_forecast) / actual)) * 100

    hw_accuracy_metrics = {
        'HoltWinters_MAE': hw_mae,
        'HoltWinters_MSE': hw_mse,
        'HoltWinters_RMSE': hw_rmse,
        'HoltWinters_MAPE': hw_mape
    }

    metrics = {
        'Bank': bank_name,
        'Correlation': correlation,
        'Min_Granger_pvalue': min(granger_pvalues.values()) if granger_pvalues else np.nan
    }
    metrics.update(arima_accuracy_metrics)
    metrics.update(arima_accuracy_metrics_no_exog)
    metrics.update(sarima_accuracy_metrics)
    metrics.update(sarima_accuracy_metrics_no_exog)
    metrics.update(hw_accuracy_metrics)

    return {
        'metrics': metrics,
        'original_price_series': original_price_series,
        'original_rate_series': original_rate_series,
        'arima_forecast_mean': arima_forecast_mean,
        'arima_conf_int': arima_conf_int,
        'arima_forecast_mean_no_exog': arima_forecast_mean_no_exog,
        'arima_conf_int_no_exog': arima_conf_int_no_exog,
        'sarima_forecast_mean': sarima_forecast_mean,
        'sarima_conf_int': sarima_conf_int,
        'sarima_forecast_mean_no_exog': sarima_forecast_mean_no_exog,
        'sarima_conf_int_no_exog': sarima_conf_int_no_exog,
        'hw_forecast': hw_forecast,
        'test_data': test_data
    }

# Create plots
def plot_bank_analysis(bank_results):
    sns.set_style('whitegrid')

    fig = plt.figure(figsize=(20, 35))

    # 1. Price and Interest Rate Time Series
    ax1 = plt.subplot(711)
    data_actual = bank_results['original_price_series']
    ax1.plot(data_actual.index, data_actual, label=f'{bank_results["metrics"]["Bank"]} Stock Price')
    ax1.set_ylabel('Stock Price')
    ax1.set_ylim([data_actual.min() * 0.9, data_actual.max() * 1.1])  # Adjust y-axis limits

    ax2 = ax1.twinx()
    ax2.plot(bank_results['original_rate_series'].index, bank_results['original_rate_series'],
             color='red', alpha=0.5, label='Interest Rate')
    ax2.set_ylabel('Interest Rate (%)', color='red')
    ax2.set_ylim([bank_results['original_rate_series'].min() * 0.9, bank_results['original_rate_series'].max() * 1.1])

    plt.title(f'{bank_results["metrics"]["Bank"]}: Stock Price vs Interest Rate')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    # 2. Scatter Plot with Trendline
    plt.subplot(712)
    x = bank_results['original_rate_series']
    y = data_actual

    plt.scatter(x, y, alpha=0.5, label='Data Points')

    # Calculate the linear regression line
    coefficients = np.polyfit(x, y, deg=1)
    polynomial = np.poly1d(coefficients)
    y_fit = polynomial(x)

    # Plot the regression line
    plt.plot(x, y_fit, color='red', label='Trend Line')

    correlation = bank_results['metrics']['Correlation']
    plt.xlabel('Interest Rate (%)')
    plt.ylabel('Stock Price')
    plt.title(f'Interest Rate vs Stock Price (Correlation: {correlation:.3f})')
    plt.legend()

    # 3. ARIMA Forecast with Exogenous Variables
    plt.subplot(713)
    arima_forecast_mean = bank_results['arima_forecast_mean']
    arima_conf_int = bank_results['arima_conf_int']
    test_data = bank_results['test_data']
    actual = bank_results['original_price_series']

    plt.plot(actual.index, actual, label='Actual')
    plt.plot(arima_forecast_mean.index, arima_forecast_mean, '--', label='ARIMA Forecast with Exogenous')
    plt.fill_between(arima_conf_int.index, arima_conf_int['lower'], arima_conf_int['upper'], color='gray', alpha=0.2, label='Confidence Interval')
    plt.ylim([actual.min() * 0.9, actual.max() * 1.1])
    plt.title(f'{bank_results["metrics"]["Bank"]}: ARIMA Forecast with Exogenous Variables')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()

    # 4. ARIMA Forecast without Exogenous Variables
    plt.subplot(714)
    arima_forecast_mean_no_exog = bank_results['arima_forecast_mean_no_exog']
    arima_conf_int_no_exog = bank_results['arima_conf_int_no_exog']

    plt.plot(actual.index, actual, label='Actual')
    plt.plot(arima_forecast_mean_no_exog.index, arima_forecast_mean_no_exog, '--', label='ARIMA Forecast without Exogenous', color='green')
    plt.fill_between(arima_conf_int_no_exog.index, arima_conf_int_no_exog['lower'], arima_conf_int_no_exog['upper'], color='gray', alpha=0.2, label='Confidence Interval')
    plt.ylim([actual.min() * 0.9, actual.max() * 1.1])
    plt.title(f'{bank_results["metrics"]["Bank"]}: ARIMA Forecast without Exogenous Variables')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()

    # 5. SARIMA Forecast with Exogenous Variables
    plt.subplot(715)
    sarima_forecast_mean = bank_results['sarima_forecast_mean']
    sarima_conf_int = bank_results['sarima_conf_int']

    plt.plot(actual.index, actual, label='Actual')
    plt.plot(sarima_forecast_mean.index, sarima_forecast_mean, '--', label='SARIMA Forecast with Exogenous')
    plt.fill_between(sarima_conf_int.index, sarima_conf_int['lower'], sarima_conf_int['upper'], color='gray', alpha=0.2, label='Confidence Interval')
    plt.ylim([actual.min() * 0.9, actual.max() * 1.1])
    plt.title(f'{bank_results["metrics"]["Bank"]}: SARIMA Forecast with Exogenous Variables')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()

    # 6. SARIMA Forecast without Exogenous Variables
    plt.subplot(716)
    sarima_forecast_mean_no_exog = bank_results['sarima_forecast_mean_no_exog']
    sarima_conf_int_no_exog = bank_results['sarima_conf_int_no_exog']

    plt.plot(actual.index, actual, label='Actual')
    plt.plot(sarima_forecast_mean_no_exog.index, sarima_forecast_mean_no_exog, '--', label='SARIMA Forecast without Exogenous', color='green')
    plt.fill_between(sarima_conf_int_no_exog.index, sarima_conf_int_no_exog['lower'], sarima_conf_int_no_exog['upper'], color='gray', alpha=0.2, label='Confidence Interval')
    plt.ylim([actual.min() * 0.9, actual.max() * 1.1])
    plt.title(f'{bank_results["metrics"]["Bank"]}: SARIMA Forecast without Exogenous Variables')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()

    # 7. Holt-Winters
    plt.subplot(717)
    hw_forecast = bank_results['hw_forecast']

    plt.plot(actual.index, actual, label='Actual')
    plt.plot(hw_forecast.index, hw_forecast, '--', label='Holt-Winters Forecast', color='purple')
    plt.ylim([actual.min() * 0.9, actual.max() * 1.1])
    plt.title(f'{bank_results["metrics"]["Bank"]}: Holt-Winters Forecast')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()

    plt.tight_layout()
    return fig

def main():
    bofa_file = 'BofA.csv'
    gs_file = 'Goldman.csv'
    jpm_file = 'JPM.csv'
    rates_file = 'Rates.csv'

    # Check if all files exist
    for file in [bofa_file, gs_file, jpm_file, rates_file]:
        if not os.path.exists(file):
            print(f"File not found: {file}")
            return

    bofa_df = load_bank_data(bofa_file)
    gs_df = load_bank_data(gs_file)
    jpm_df = load_bank_data(jpm_file)
    rates_df = load_rates_data(rates_file)

    start_date = '2000-01-03'
    if bofa_df is not None:
        bofa_df = bofa_df[bofa_df.index >= start_date]
    if gs_df is not None:
        gs_df = gs_df[gs_df.index >= start_date]
    if jpm_df is not None:
        jpm_df = jpm_df[jpm_df.index >= start_date]
    if rates_df is not None:
        rates_df = rates_df[rates_df.index >= start_date]

    banks = {
        'Bank of America': bofa_df,
        'Goldman Sachs': gs_df,
        'JPMorgan Chase': jpm_df
    }

    metrics_summary = []
    results = {}

    for bank_name, bank_df in banks.items():
        print(f"\n=== Analyzing {bank_name} ===")
        analysis_result = analyze_bank_rates_relationship(bank_df, rates_df, bank_name)

        if analysis_result is None:
            print(f"Analysis failed for {bank_name}.")
            continue

        # Extract and store metrics
        metrics = analysis_result['metrics']
        metrics_summary.append(metrics)

        # Store data for plotting
        results[bank_name] = analysis_result

    if not metrics_summary:
        print("No successful analyses to plot or summarize.")
        return

    # Print Accuracy Metrics Grouped by Bank
    for metrics in metrics_summary:
        print(f"\n==== {metrics['Bank']} Accuracy Metrics ====")
        print(f"Correlation with Interest Rates: {metrics['Correlation']:.6f}")
        if not np.isnan(metrics['Min_Granger_pvalue']):
            print(f"Minimum Granger Causality p-value: {metrics['Min_Granger_pvalue']:.6f}")
        else:
            print("Granger Causality test was not performed due to insufficient data.")

        print("\nARIMA Forecast Accuracy Metrics (with exogenous variables):")
        print(f"  MAE: {metrics['ARIMA_MAE_with_exog']:.2f}")
        print(f"  MSE: {metrics['ARIMA_MSE_with_exog']:.2f}")
        print(f"  RMSE: {metrics['ARIMA_RMSE_with_exog']:.2f}")
        print(f"  MAPE: {metrics['ARIMA_MAPE_with_exog']:.2f}%")

        print("\nARIMA Forecast Accuracy Metrics (without exogenous variables):")
        print(f"  MAE: {metrics['ARIMA_MAE_no_exog']:.2f}")
        print(f"  MSE: {metrics['ARIMA_MSE_no_exog']:.2f}")
        print(f"  RMSE: {metrics['ARIMA_RMSE_no_exog']:.2f}")
        print(f"  MAPE: {metrics['ARIMA_MAPE_no_exog']:.2f}%")

        print("\nSARIMA Forecast Accuracy Metrics (with exogenous variables):")
        print(f"  MAE: {metrics['SARIMA_MAE_with_exog']:.2f}")
        print(f"  MSE: {metrics['SARIMA_MSE_with_exog']:.2f}")
        print(f"  RMSE: {metrics['SARIMA_RMSE_with_exog']:.2f}")
        print(f"  MAPE: {metrics['SARIMA_MAPE_with_exog']:.2f}%")

        print("\nSARIMA Forecast Accuracy Metrics (without exogenous variables):")
        print(f"  MAE: {metrics['SARIMA_MAE_no_exog']:.2f}")
        print(f"  MSE: {metrics['SARIMA_MSE_no_exog']:.2f}")
        print(f"  RMSE: {metrics['SARIMA_RMSE_no_exog']:.2f}")
        print(f"  MAPE: {metrics['SARIMA_MAPE_no_exog']:.2f}%")

        print("\nHolt-Winters Forecast Accuracy Metrics:")
        print(f"  MAE: {metrics['HoltWinters_MAE']:.2f}")
        print(f"  MSE: {metrics['HoltWinters_MSE']:.2f}")
        print(f"  RMSE: {metrics['HoltWinters_RMSE']:.2f}")
        print(f"  MAPE: {metrics['HoltWinters_MAPE']:.2f}%")

    # Plotting for each bank
    for bank_name, analysis_result in results.items():
        print(f"\nGenerating plots for {bank_name}...")
        fig = plot_bank_analysis(analysis_result)
        fig_path = f'{bank_name.replace(" ", "_")}_analysis.png'
        plt.savefig(fig_path)
        plt.close()
        print(f"Plot saved to {fig_path}")


    return metrics_summary

main()




Successfully read BofA.csv
Duplicate dates found in index. Handling duplicates...
Successfully read Goldman.csv
Some 'Pricing Date' values could not be converted to datetime.
Successfully read JPM.csv
Some 'Pricing Date' values could not be converted to datetime.
Successfully read Rates.csv

=== Analyzing Bank of America ===
Date range of merged_df: 2000-01-03 00:00:00 to 2024-11-26 00:00:00
Correlation between Share Price and MORTGAGE30US: 0.43030396341548355

=== Stationarity Test for Bank of America Price ===
ADF Statistic: -1.251355864919427
p-value: 0.6511231082286751
Critical values:
	1%: -3.431360019071387
	5%: -2.8619863416168188
	10%: -2.567007584327379
Bank of America Price is non-stationary. Differencing is recommended.

=== Stationarity Test for Interest Rate ===
ADF Statistic: -2.1577118851367367
p-value: 0.22194985197955258
Critical values:
	1%: -3.4313625212742105
	5%: -2.8619874471870737
	10%: -2.567008172846683
Interest Rate is non-stationary. Differencing is recommend

[{'Bank': 'Bank of America',
  'Correlation': 0.43030396341548355,
  'Min_Granger_pvalue': 0.93002162615197,
  'ARIMA_MAE_with_exog': 5.508448563437514,
  'ARIMA_MSE_with_exog': 43.6516929220888,
  'ARIMA_RMSE_with_exog': 6.6069427818083,
  'ARIMA_MAPE_with_exog': 17.16884212900028,
  'ARIMA_MAE_no_exog': 6.135168407512112,
  'ARIMA_MSE_no_exog': 52.063804033351126,
  'ARIMA_RMSE_no_exog': 7.21552520842046,
  'ARIMA_MAPE_no_exog': 19.997500791521382,
  'SARIMA_MAE_with_exog': 5.57609241143935,
  'SARIMA_MSE_with_exog': 44.61720311258997,
  'SARIMA_RMSE_with_exog': 6.679611000094988,
  'SARIMA_MAPE_with_exog': 17.566810030567556,
  'SARIMA_MAE_no_exog': 5.443962674596028,
  'SARIMA_MSE_no_exog': 42.71276426435557,
  'SARIMA_RMSE_no_exog': 6.535500307119231,
  'SARIMA_MAPE_no_exog': 16.995515269550136,
  'HoltWinters_MAE': 5.545339631051432,
  'HoltWinters_MSE': 44.400231230560216,
  'HoltWinters_RMSE': 6.663349850530153,
  'HoltWinters_MAPE': 17.79611205011175},
 {'Bank': 'Goldman Sachs